In [ ]:
from random import randint
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D
from tensorflow.keras.layers import Flatten, MaxPool2D, AvgPool2D, BatchNormalization 
from tensorflow.keras.regularizers import l2 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

df_treino_2017 = pd.read_csv('isic_2017_treino.csv')
df_treino_2018 = pd.read_csv('isic_2018_treino.csv')

# ctrl c ctrl v
i, j, chnls = 200, 200, 3

cnn = Sequential()
cnn.add(Conv2D(50, (3, 3), input_shape = (i, j, chnls), activation = 'relu'))
cnn.add(Conv2D(50, (3, 3), input_shape = (i, j, chnls), activation = 'relu'))
cnn.add(MaxPool2D((2, 2)))
cnn.add(Conv2D(50, (3, 3), input_shape = (i, j, chnls), activation = 'relu'))
cnn.add(Conv2D(50, (3, 3), input_shape = (i, j, chnls), activation = 'relu'))
cnn.add(AvgPool2D((2, 2)))
cnn.add(Conv2D(50, (3, 3), input_shape = (i, j, chnls), activation = 'relu'))
cnn.add(BatchNormalization())
cnn.add(AvgPool2D((2, 2)))
cnn.add(Flatten())
cnn.add(Dense(25, activation = 'relu', kernel_regularizer = l2(0.05)))
cnn.add(Dropout(0.2))
cnn.add(Dense(25, activation = 'relu', kernel_regularizer = l2(0.05)))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.2))
cnn.add(Dense(25, activation = 'relu', kernel_regularizer = l2(0.05)))
cnn.add(Dropout(0.2))
cnn.add(Dense(25, activation = 'relu', kernel_regularizer = l2(0.05)))
cnn.add(Dense(1, activation = 'sigmoid'))
        
cnn.compile(loss = 'binary_crossentropy', optimizer = RMSprop(learning_rate = 0.001))

es = EarlyStopping(monitor = 'loss', patience = 5) 

In [ ]:
# FUNÇÕES

def get_images_n_labels(dataframe, i, j, path = '17n18_train/', series_name = 'img', label_name = 'pos'):
    
    x = []
    y = []
    
    for n in range(dataframe.shape[0]):
        img = cv2.imread(path + dataframe[series_name].iloc[n])
        img = cv2.resize(img, (i, j))
        
        x.append(img)
        y.append(dataframe[label_name].iloc[n])
        
    return np.array(x), np.array(y)

# https://stackoverflow.com/questions/25008458/how-to-apply-clahe-on-rgb-color-images
def bgr_CLAHE(img):
    

    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit = 2.0,tileGridSize = (6, 6))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    
    return img

# daqui para baixo, funções referentes ao processo de data augmentation
# 'zoom' recorte seguido de resize
def zoom(img, original_dim = [200, 200], h_slice = [10, 190], v_slice = [10, 190]):

    img = img[v_slice[0] : v_slice[1], h_slice[0] : h_slice[1]]
    img = cv2.resize(img, (original_dim[0], original_dim[1]))
    
    return img 

# horizontal shift
def h_shift(image, original_dim = [200, 200], shift = 10):
    
    T_x = shift
    T_y = 0
    
    M = np.array([[1, 0, T_x], [0, 1, T_y]], dtype = 'float32')
    img_transladada = cv2.warpAffine(image, M, (200, 200))
    img = img_transladada[0 : original_dim[0], shift : original_dim[1]]
    img = cv2.resize(img, (original_dim[0], original_dim[1]))
    
    return img

# vertical shift
def v_shift(image, original_dim = [200, 200], shift = 10):
    
    T_x = 0
    T_y = shift
    
    M = np.array([[1, 0, T_x], [0, 1, T_y]], dtype = 'float32')
    img_transladada = cv2.warpAffine(image, M, (200, 200))
    img = img_transladada[shift : original_dim[0], 0 : original_dim[1]]
    img = cv2.resize(img, (original_dim[0], original_dim[1]))
    
    return img

def rotation_90(img):

    rows, cols, chnls = img.shape
    M = cv2.getRotationMatrix2D(((cols - 1) / 2.0, (rows - 1) / 2.0), 90, 1)
    img = cv2.warpAffine(img, M, (cols, rows))
    
    return img

def data_augmentation(x, y):
    
    augmentation_imgs = []
    augmentation_labels = []

    for n in range(len(x)):

        image = x[n]
        classe = y[n]
        augment = randint(0, 1) # booleano (50% de chance de aplicar augmentation)

        if augment == 1:
            process = randint(0, 5) # seleção aleatória do processo de augmentation

            if process == 0:
                image = cv2.flip(image, 0)
                augmentation_imgs.append(image) # horizontal flip
                augmentation_labels.append(classe)

            if process == 1:
                image = zoom(image)
                augmentation_imgs.append(image) # zoom 0.2
                augmentation_labels.append(classe)

            if process == 2:
                image = h_shift(image)
                augmentation_imgs.append(image) # horizontal shift
                augmentation_labels.append(classe)

            if process == 3:
                image = v_shift(image)
                augmentation_imgs.append(image) # vertical shift
                augmentation_labels.append(classe)

            if process == 4:
                image = rotation_90(image)
                augmentation_imgs.append(image) # rotaton 90°
                augmentation_labels.append(classe)
                
    return augmentation_imgs, augmentation_labels

# este processo deve rodar uma vez com cada um dos batches acima, logo em seguida, o resultado da operação
# deve passar pela rede
def single_batch_prep(batch_df, width, height):
    # normalização ?
    x, y = get_images_n_labels(batch_df, width, height)

    # pré processamento de imagens
    x = [bgr_CLAHE(i) for i in x]
    x = [cv2.fastNlMeansDenoisingColored(i) for i in x]
#     x = [cv2.cvtColor(i, cv2.COLOR_BGR2GRAY) for i in x] # precisa fazer isso ?

    # data augmentation
    augmentation_imgs, augmentation_labels = data_augmentation(x, y)
    
    return np.array(x + augmentation_imgs), np.array(list(y) + augmentation_labels) 

In [ ]:
# TRATAMENTO

# tratando 2017

# adicionando extensão aos nomes dos arquivos
df_treino_2017['image_id'] = df_treino_2017['image_id'].apply(lambda x: x + '.jpg')

# criando flag para nevus
df_treino_2017['nevus'] = ((df_treino_2017.melanoma == 0) & 
                           (df_treino_2017.seborrheic_keratosis == 0)).astype(float)

df_treino_2017.drop('seborrheic_keratosis', axis = 1, inplace = True)

# as imagens precisam ser ou nevus ou melanoma
df_treino_2017 = df_treino_2017.loc[(df_treino_2017['melanoma'] == 1) | (df_treino_2017['nevus'] == 1)]

# minha pasta 'isic_2017_treino' possui 450 imagens (algumas destas, não são nevus ou melanoma)

dir_treino_17 = r'C:\Users\fkhon\Documents\LETSCODE\MODULO9\projeto_m9_1311\isic_2017_treino'
dir_treino_17_filenames = [name for name in os.listdir(dir_treino_17)]

# eliminarei do descritivo, os nomes que não constam na pasta 'isic_2017_treino'
df_treino_2017 = \
  df_treino_2017.loc[df_treino_2017['image_id'].isin(dir_treino_17_filenames)]

# tratando 2018
# tratando nomes (adicionando extensões aos nomes) e filtrando series 
df_treino_2018 = df_treino_2018[['image', 'MEL', 'NV']]
df_treino_2018['image'] = df_treino_2018['image'].apply(lambda x: x + '.jpg')

# verificando correspondência entre descritivo e pasta
dir_treino_18 = r'C:\Users\fkhon\Documents\LETSCODE\MODULO9\projeto_m9_1311\isic_2018_treino'
dir_treino_18_filenames = [name for name in os.listdir(dir_treino_18)]

# imagens precisam ser ou nevus ou melanoma
df_treino_2018 = df_treino_2018.loc[(df_treino_2018['MEL'] == 1) | (df_treino_2018['NV'] == 1)]

# unificar nomes das series, para eventual concat
df_treino_2018.columns = ['img', 'pos', 'neg']
df_treino_2017.columns = ['img', 'pos', 'neg']

# juntar as duas bases, a fim de somar 1200 imagens de cada caso
# para tanto, somarei os casos positivos de 2017 aos de 2018, e eliminarei de 2018, casos negativos
# suficientes para que o total se equipare ao de positivos

# separando casos positivos e negativos de cada ano
pos_2018 = df_treino_2018.loc[df_treino_2018['pos'] == 1]
pos_2017 = df_treino_2017.loc[df_treino_2017['pos'] == 1]

neg_2018 = df_treino_2018.loc[df_treino_2018['neg'] == 1]
neg_2017 = df_treino_2017.loc[df_treino_2017['neg'] == 1]

# juntando casos positivos e negativos de todos os anos
full_pos = pd.concat([pos_2017, pos_2018], axis = 0)
full_neg = pd.concat([neg_2017, neg_2018], axis = 0)

# amostrando aleatoriamente, 1204 dados negativos, estes consistirão nas observações negativas (a amostragem
# é feita para separar um número de imagens equivalente ao número de imagens positivas que temos à disposição)
full_neg = full_neg.sample(n = full_pos.shape[0], random_state = 123)

full_pos = full_pos[['img', 'pos']]
full_neg = full_neg[['img', 'pos']]

filenames = pd.concat([full_pos, full_neg]).reset_index().drop('index', axis = 1)

sample = filenames.sample(150)
sample = sample.reset_index()
sample = sample.drop('index', axis = 1)

In [ ]:
# VALIDAÇÃO (extração e pré processamento)

df_validacao_2017 = pd.read_csv('isic_2017_validacao.csv')
df_validacao_2017.drop('seborrheic_keratosis', axis = 1, inplace = True)
df_validacao_2017.columns = ['img', 'pos']
df_validacao_2017['img'] = df_validacao_2017['img'].apply(lambda x: x + '.jpg')

# tirando slice, para ser mais rápido
df_validacao_2017.sample(50, random_state = 123)

x_val, y_val = get_images_n_labels(df_validacao_2017, 200, 200, path = 'isic_2017_validacao/')

# pré processamento de imagens
x_val = [bgr_CLAHE(i) for i in x_val]
x_val = [cv2.fastNlMeansDenoisingColored(i) for i in x_val]

In [ ]:
# daqui para baixo, 'sample' representará o dataframe contendo os nomes de todas as imagens que serão 
# passadas pela rede

# começando já na próxima célula, devemos fracionar os dados (batches)
# a função 'get_images_n_labels()' deverá receber frações do dataframe, estas que serão pré processadas e 
# passarão pelo processo de augmentation em 'turnos'

In [ ]:
# AQUI: fracionar dados, para que possam ser passados pelo processo abaixo em 'batches'

batches = 5
step = sample.shape[0] // batches

count = 0
lower_idx = 0
dataframe = sample

for n in range(batches):
    
    upper_idx = lower_idx + step 
    
    exec(f'batch_{count} = dataframe.iloc[{lower_idx}:{upper_idx}]')
    
    count += 1
    lower_idx += step
    
# aqui, realizamos o fracionamento do dataframe contendo os nomes e labels das imagens
# exemplo:
# batch_0 = dataframe.iloc[0:30]
# batch_1 = dataframe.iloc[30:60]
# batch_2 = dataframe.iloc[60:90]
# batch_3 = dataframe.iloc[90:120]
# batch_4 = dataframe.iloc[120:150]

In [ ]:
# retorna batches (com base nos dataframes, deta deve rodar uma vez para cada dataframe)
x_0, y_0 = single_batch_prep(batch_0, 200, 200)
x_1, y_1 = single_batch_prep(batch_1, 200, 200)
# x_2, y_2 = single_batch_prep(batch_2, 200, 200)
# x_3, y_3 = single_batch_prep(batch_3, 200, 200)
# x_4, y_4 = single_batch_prep(batch_4, 200, 200)

In [ ]:
# acho que aqui, teríamos a rede, esta receberia uma porção de dados recém extraída, pré processada e 
# 'aumentada', então, após o ajuste a porção em questão, faríamos a extração, pré processamento e 'aumento'
# de outra porção

In [ ]:
# treinando no batch_0
cnn.fit(x = x_0, y = y_0, epochs = 50, callbacks = [es], batch_size = 3 , validation_data = (np.array(x_val),
                                                                                             y_val))

In [ ]:
# treinando no batch_1
cnn.fit(x = x_1, y = y_1, epochs = 50, callbacks = [es], batch_size = 3, validation_data = (np.array(x_val), 
                                                                                            y_val))